In [1]:
from urllib import request
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
from ztfquery import marshal
import numpy as np
import csv
from astropy.time import Time
import datetime
from datetime import timedelta 
from matplotlib import pyplot as plt
from astroplan import Observer, FixedTarget
from astropy.coordinates import SkyCoord,EarthLocation
import astropy.units as u
from astroplan.plots import plot_airmass
from astroplan import AirmassConstraint, is_event_observable, AtNightConstraint
import time
import os


>>> from astroplan import download_IERS_A
>>> download_IERS_A()
 [astroplan.utils]


In [2]:
def estimate_error_r(time,mag,err,now):
    mags_now=[]
    for n in range(1,1000):
        mgr1fit_temp=err * np.random.randn(len(err)) + mag
        rfit,cr = np.polyfit(jdr1,mgr1fit_temp,1,cov=True,w=mgr1_err)        
        p=np.poly1d(rfit) 
        mags_now.append(p(now)) 
    return np.mean(mags_now),np.std(mags_now) 

def estimate_error_g(time,mag,err,now):
    mags_now=[]
    for n in range(1,1000):
        mgg1fit_temp=err * np.random.randn(len(err)) + mag
        gfit,cg = np.polyfit(jdg1,mgg1fit_temp,1,cov=True,w=mgg1_err)
        p=np.poly1d(gfit)
        mags_now.append(p(now))
    return np.mean(mags_now),np.std(mags_now)

  # Get an object list from url
  You may change the acount and url given below. The url below is for type Ib,Ic and IIb SN. Here we only select those whose redshift is smaller than 0.05.

In [3]:

username = ''
password = ''
url='http://skipper.caltech.edu:8080/cgi-bin/growth/list_sources.cgi?search=all+targets&candtypes=AnyType&finalcandtypes=AnyType&classes=Ic&classes=Ic-BL&classes=Ic-BL%3F&classes=IIb&classes=IIbn%3F&classes=SN+Ib&classes=SN+Ib%3F&classes=SN+Ib%2Fc&classes=SN+Ib%2Fc%3F&classes=SNIb%2FIIb%3F&classes=SN+Ibn&classes=SN+Ibn%3F&classes=SNIc&classes=SN+Ic&classes=SN+Ic%3F&classes=SNIc%3F&classes=SN+Ic-BL&classes=SN+Ic-broad%3F&classes=SN+Ic-norm&classes=SN+IIb&classes=SN+IIb%3F&sortby=datesaved&reverse=1&nshow=100&datetype=Saved&fromyear=2000&frommonth=Jul&fromday=27&toyear=2019&tomonth=Aug&today=27&fromra=0.0&tora=360.0&fromdec=-90.0&todec=90.0&runs=-1&fromrunpriority=0&ogroups=-1&fupprogram=-1&frompriority=0&commentlogic=&comment=&postedby=&tabletype=starlist&plotlc=yes&doit=yes'

passman = urllib.request.HTTPPasswordMgrWithDefaultRealm()
passman.add_password(None, url, username, password)
handler = urllib.request.HTTPBasicAuthHandler(passman)
opener = urllib.request.build_opener(handler)
x = opener.open(url)
soup = BeautifulSoup(x)
strtable = str(soup)
objraw = re.findall ('ZTF\d\d\D{0,7}', strtable)

In [4]:
m = marshal.MarshalAccess()
data = m.get_target_data(objraw)
datalist = data.reset_index()
rs = m.get_target_redshift(objraw)
rslist = rs.reset_index()
nlist = []
rlist = []
clist = []
for i in rslist.index:
    if rslist.redshift[i] != None and rslist.redshift[i] < str(0.05):
        nlist.append(datalist.name[i])
        clist.append(datalist.classification[i])
        rlist.append(rslist.redshift[i])

d = {'name': nlist,'classification':clist,'redshift': rlist}
df = pd.DataFrame(data=d)

INFO: I am downloading target_sources metadata, it may take some time (~1min). 
You may fasten this part by first doing self.load_target_sources(YOUR_PROGRAM)


In [5]:
#GET JDNOW
now = datetime.datetime.now()
jdnow = Time(now)

# Get discovery date and time of latest spectra
Here we select those who are older than 150 days.

In [6]:
obj = df['name']

ddlist = []
deltatlist1 = []
lslist= []
deltatlist2 =[]

for x in obj:
    try:
        marshal.download_lightcurve(str(x),overwrite=True)
        lc_dict = marshal.get_local_lightcurves(str(x))
        lcjd = lc_dict.jdobs[lc_dict.mag != 99.00].reset_index()
        lcd0 = lc_dict.date[lc_dict.mag!=99.00].reset_index()
        ddlist.append(lcd0.date[0]) #discovery date
        t1 = round(jdnow.jd-lcjd.jdobs[0],2)
        deltatlist1.append(t1)
    except Exception:
        ddlist.append("cannot access")
        deltatlist1.append("cannot access")
    try:
        marshal.download_spectra(str(x),overwrite=True)
        spectra = marshal.get_local_spectra(str(x))
        specdata = list(spectra)
        specdata.sort()
        spdate = specdata[-1]       
        dstrip = re.search('\d{7,8}',spdate)
        time_k = time.mktime(time.strptime(dstrip.group(),'%Y%m%d'))
        time_kk = time.localtime(time_k)
        t_form = time.strftime('%Y %b %d',time_kk)
        lslist.append(t_form)
        t_form_u = time.strftime('%Y-%m-%d',time_kk)
        jdspec = Time(t_form_u)
        t2= round(jdspec.jd-lcjd.jdobs[0],2)
        deltatlist2.append(t2)
    except Exception:
        lslist.append("cannot access")
        deltatlist2.append("cannot access")
                
df['discovery date']=ddlist
df['age/days']=deltatlist1
df['date of latest spectra']=lslist
df['time interval(latest spectra to discovery)/days']=deltatlist2

df_acc=df[df['age/days']!='cannot access']
df_new=df_acc[df_acc['age/days']>150].reset_index(drop=True)

/Users/yuhsiao/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# Estimate current magnitude (r & g)

In [7]:
rflist=[]
rulist=[]
gflist=[]
gulist=[]   

obj2=df_new['name']

for x in obj2:
    try:
        marshal.download_lightcurve(str(x),overwrite=True)
        lc_dict = marshal.get_local_lightcurves(str(x))   
        jdlis = lc_dict.jdobs[lc_dict.mag != 99.00]
        maglis = lc_dict.mag[lc_dict.mag!=99.00]
        emaglis = lc_dict.emag[lc_dict.mag!=99.00]

        jdr_old = jdlis[lc_dict['filter']=='r'].reset_index()
        jdr = jdr_old.jdobs
        jdg_old = jdlis[lc_dict['filter']=='g'].reset_index()
        jdg = jdg_old.jdobs
        mgr_old = maglis[lc_dict['filter']=='r'].reset_index()
        mgr = mgr_old.mag
        mgg_old = maglis[lc_dict['filter']=='g'].reset_index()
        mgg = mgg_old.mag
        mgr_err_old = emaglis[lc_dict['filter']=='r'].reset_index()
        mgr_err = mgr_err_old.emag
        mgg_err_old = emaglis[lc_dict['filter']=='g'].reset_index()
        mgg_err = mgg_err_old.emag   
            
        if jdr.values.size == 0:
            rflist.append("Cannot fit")
            rulist.append("Cannot fit")
        elif jdr.values.size > 0:
            dis_r_jd = float(jdr.values[0])
            late_r_jd = float(jdr.values[-1])
            jdr1 = jdr[jdr>= late_r_jd-50]
            mgr1 = mgr[jdr>= late_r_jd-50]
            mgr1_err = mgr_err[jdr >= late_r_jd-50]
            if jdr1.values.size <3:
                rflist.append("cannot fit")
                rulist.append("cannot fit")
            elif jdr1.values.size >2:
                rfit,cr = np.polyfit(jdr1,mgr1,1,cov=True,w=mgr1_err)     
                p=np.poly1d(rfit)
                magnow,err=estimate_error_r(jdr1,mgr1,mgr1_err,jdnow.jd)
                rflist.append(round(magnow,2))
                rulist.append(round(err,2))

        if jdg.values.size == 0:
            gflist.append("Cannot fit")
            gulist.append("Cannot fit")
        elif jdg.values.size > 0:
            dis_g_jd = float(jdg.values[0])
            late_g_jd = float(jdg.values[-1])
            jdg1 = jdg[jdg>= late_g_jd-50]
            mgg1 = mgg[jdg>= late_g_jd-50]
            mgg1_err = mgg_err[jdg>= late_g_jd-50]
                
            if jdg1.values.size <3:
                gflist.append("cannot fit")
                gulist.append("cannot fit")
            elif jdg1.values.size >2:
                gfit,cg = np.polyfit(jdg1,mgg1,1,cov=True,w=mgg1_err)
                p=np.poly1d(gfit)
                magnow,err=estimate_error_g(jdg1,mgg1,mgg1_err,jdnow.jd)
                gflist.append(round(magnow,2))
                gulist.append(round(err,2))
    except Exception:
        rflist.append("cannot access")
        rulist.append("cannot access")
        gflist.append("cannot access")
        gulist.append("cannot access")
            
df_new["mag estimated (r)"]= rflist
df_new["mag uncertainty (r)"]=rulist
df_new["mag estimated (g)"]=gflist
df_new["mag uncertainty (g)"]=gulist


# Caculate observable time
Here we select those who are observable for more than 15 minutes.

In [8]:
obtlist=[]
obdtlist=[]

observe_time = jdnow + np.linspace(0, 24, 100)*u.hour
keck = Observer.at_site("Keck Observatory",timezone="US/Hawaii")
#m = marshal.MarshalAccess()
data = m.get_target_data(obj2)
datalist = data.reset_index(drop=True)
coord0 = m.get_target_coordinates(obj2)
colist = coord0.reset_index(drop=True)

for i in colist.index:
    name = datalist["name"][i]
    ra = colist["ra"][i]
    dec = colist["dec"][i]    
    coord = SkyCoord(ra= ra*u.deg,dec= dec*u.deg)
    target = FixedTarget(coord= coord, name=name)  
    constraints = [AirmassConstraint(2),AtNightConstraint.twilight_civil()]
    isobservable = is_event_observable(constraints,keck,target,times=observe_time)
    isob = isobservable.flatten()
    obtime0 = observe_time.to_datetime()
    obtime = obtime0[isob==True]
    obtimeF = obtime0[isob==False]
    
    if isob[isob==True].size==0:
        obtlist.append("cannot be observed")
        obdtlist.append("cannot be observed")
    elif isob[0]==False:
        obtrange=[obtime[0].strftime("%Y-%m-%d %H:%M:%S") ,obtime[-1].strftime("%Y-%m-%d %H:%M:%S")]
        obtlist.append(obtrange)
        deltat=obtime[-1]-obtime[0]
        
        minutes_total, seconds = divmod(deltat.seconds, 60)
        hours, minutes = divmod(minutes_total, 60) 
        deltat_str = '%d:%02d:%02d' % (hours, minutes, seconds)
        t = time.strptime(deltat_str,"%H:%M:%S") 
        deltat_format = timedelta(hours=t.tm_hour, minutes=t.tm_min, seconds=t.tm_sec) 
        obdtlist.append(deltat_format)
    
    elif isob[0]==True:
        obtrange1=[obtime0[0].strftime("%Y-%m-%d %H:%M:%S") ,obtimeF[0].strftime("%Y-%m-%d %H:%M:%S")] 
        obtrange2=[obtimeF[-1].strftime("%Y-%m-%d %H:%M:%S") ,obtime0[-1].strftime("%Y-%m-%d %H:%M:%S")] 
        obtrange3=[obtrange1,obtrange2]
        obtlist.append(obtrange3)
        deltat1=obtimeF[0]-obtime0[0]
        deltat2=obtime0[-1]-obtimeF[-1]
        deltat3=deltat1+deltat2
        
        min_total, sec = divmod(deltat3.seconds, 60)
        hour, minu = divmod(min_total, 60) 
        deltat3_str = '%d:%02d:%02d' % (hour, minu, sec)
        t3 = time.strptime(deltat3_str,"%H:%M:%S") 
        deltat3_format = timedelta(hours=t3.tm_hour, minutes=t3.tm_min, seconds=t3.tm_sec) 
        obdtlist.append(deltat3_format)

df_new["observable time"]= obtlist
df_new["observable hours"]=obdtlist

df_ob=df_new[df_new["observable hours"] != 'cannot be observed' ]
df_ob_new=df_ob[df_ob["observable hours"]>timedelta(minutes=15)].reset_index(drop=True)
df_ob_new.to_csv('ob_plan.csv')     